In [3]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq

from matplotlib import pyplot as plt
import seaborn as sns

import sys
sys.path.append('../../guchio_utils/')

In [4]:
#plt.rcParams['font.family'] = 'IPAPGothic' #全体のフォントを設定
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%config InlineBackend.figure_formats = {'png', 'retina'}
%load_ext autoreload
%autoreload 1

In [5]:
#trn_df = pd.read_parquet('../inputs/origin/train.parquet.zip', compression='zip')
trn_df = pq.read_pandas('../inputs/origin/train.parquet').to_pandas()
trn_meta_df = pd.read_csv('../inputs/origin/metadata_train.csv')

In [6]:
trn_df

,0,1,2,3,4,5,6,7,8,9,...,8702,8703,8704,8705,8706,8707,8708,8709,8710,8711
0,18,1,-19,-16,-5,19,-15,15,-1,-16,...,18,-22,12,8,13,6,-21,-15,-9,20
1,18,0,-19,-17,-6,19,-17,16,0,-15,...,17,-21,12,8,14,7,-19,-15,-8,21
2,17,-1,-20,-17,-6,19,-17,15,-3,-15,...,16,-21,13,8,15,8,-18,-14,-8,22
3,18,1,-19,-16,-5,20,-16,16,0,-15,...,16,-21,12,8,15,8,-19,-14,-7,23
4,18,0,-19,-16,-5,20,-17,16,-2,-14,...,17,-22,12,8,15,8,-18,-14,-8,23
5,18,0,-20,-15,-4,20,-16,15,-2,-12,...,17,-22,12,8,15,8,-19,-14,-8,22
6,19,1,-18,-16,-5,20,-16,16,-1,-13,...,18,-22,12,8,15,8,-19,-14,-8,22
7,18,0,-19,-17,-7,18,-17,15,-2,-15,...,17,-22,11,7,13,6,-20,-14,-8,22
8,18,0,-20,-18,-7,18,-16,15,-2,-16,...,17,-23,11,8,13,6,-21,-14,-8,22
9,17,0,-19,-17,-7,18,-17,17,-1,-14,...,16,-22,11,7,13,6,-21,-14,-8,22


In [7]:
trn_meta_df

,signal_id,id_measurement,phase,target
0,0,0,0,0
1,1,0,1,0
2,2,0,2,0
3,3,1,0,1
4,4,1,1,1
5,5,1,2,1
6,6,2,0,0
7,7,2,1,0
8,8,2,2,0
9,9,3,0,0


In [8]:
trn_df.info(), trn_meta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Columns: 8712 entries, 0 to 8711
dtypes: int8(8712)
memory usage: 6.5 GB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8712 entries, 0 to 8711
Data columns (total 4 columns):
signal_id         8712 non-null int64
id_measurement    8712 non-null int64
phase             8712 non-null int64
target            8712 non-null int64
dtypes: int64(4)
memory usage: 272.3 KB


(None, None)

## basic EDA 

### やりたいこと
 * noise 除去
     * HPF
     * LPH
 * FFT 周りでなにか

In [9]:
def plt_phase_current():

SyntaxError: unexpected EOF while parsing (<ipython-input-9-3e243f4997f0>, line 1)

In [23]:
trn_meta_df.sample(100000, replace=True)

,signal_id,id_measurement,phase,target
8106,8106,2702,0,0
5133,5133,1711,0,0
3435,3435,1145,0,0
6033,6033,2011,0,0
7087,7087,2362,1,0
5762,5762,1920,2,0
5280,5280,1760,0,0
5213,5213,1737,2,0
3310,3310,1103,1,1
5172,5172,1724,0,0


In [24]:
from imblearn.over_sampling import SMOTE

In [55]:
trn_meta_df.head()

,signal_id,id_measurement,phase,target
0,0,0,0,0
1,1,0,1,0
2,2,0,2,0
3,3,1,0,1
4,4,1,1,1


In [67]:
trn_meta_df['temp'] = [np.random.rand() for _ in range(len(trn_meta_df.index))]
trn_meta_df

,signal_id,id_measurement,phase,target,temp
0,0,0,0,0,0.401314
1,1,0,1,0,0.649012
2,2,0,2,0,0.846979
3,3,1,0,1,0.690238
4,4,1,1,1,0.984395
5,5,1,2,1,0.965821
6,6,2,0,0,0.500915
7,7,2,1,0,0.964628
8,8,2,2,0,0.833870
9,9,3,0,0,0.096627


In [72]:
res = SMOTE().fit_resample(trn_meta_df.drop('target', axis=1), [trn_meta_df[['target', 'id_measurement']]])
res[0].shape, res[1].shape

/home/naoya.taguchi/anaconda3/lib/python3.7/site-packages/imblearn/utils/_validation.py:93: UserWarning: 'y' should be of types ('binary', 'multiclass', 'multilabel-indicator') only. Got unknown instead.
  TARGET_KIND, type_of_target(y)))


ValueError: cannot copy sequence with size 8712 to array axis with dimension 2

In [71]:
pd.DataFrame(res[0])#.describe()

,0,1,2,3
0,0.000000,0.000000,0.000000,0.401314
1,1.000000,0.000000,1.000000,0.649012
2,2.000000,0.000000,2.000000,0.846979
3,3.000000,1.000000,0.000000,0.690238
4,4.000000,1.000000,1.000000,0.984395
5,5.000000,1.000000,2.000000,0.965821
6,6.000000,2.000000,0.000000,0.500915
7,7.000000,2.000000,1.000000,0.964628
8,8.000000,2.000000,2.000000,0.833870
9,9.000000,3.000000,0.000000,0.096627
